In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import re
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

In [2]:
from tqdm import tqdm


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [4]:
import warnings

In [5]:
# Load data using relative paths
raw_data = pd.read_csv("C:/Users/e.sansebastian\OneDrive - EXPONDO/Ironhack/Weeks/Week_9/Final_project/final_project/data/data_uncleaned.csv")

In [6]:
raw_data.head()

,Unnamed: 0,review_title,review_rating,review_content,source
0,0,"TIG (WIG) welder, model S-WIGMA 200PDC",5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",expondo
1,1,Awesome experience,5,Awesome experience. We had to use their custom...,expondo
2,2,The product arrived on time all the…,5,The product arrived on time all the assembly d...,expondo
3,3,Good and fast delivery,5,Good and fast delivery of this amazing standin...,expondo
4,4,Bad invoice cross-border purchase - no contact...,1,Bad invoice - DPH on invoice - shipping to slo...,expondo


In [7]:
raw_data["review_rating"] = raw_data["review_rating"].astype(str)
raw_data['review_id'] = raw_data.index.astype(str)
display(raw_data.head())
display(raw_data.shape)

,Unnamed: 0,review_title,review_rating,review_content,source,review_id
0,0,"TIG (WIG) welder, model S-WIGMA 200PDC",5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",expondo,0
1,1,Awesome experience,5,Awesome experience. We had to use their custom...,expondo,1
2,2,The product arrived on time all the…,5,The product arrived on time all the assembly d...,expondo,2
3,3,Good and fast delivery,5,Good and fast delivery of this amazing standin...,expondo,3
4,4,Bad invoice cross-border purchase - no contact...,1,Bad invoice - DPH on invoice - shipping to slo...,expondo,4


(32271, 6)

In [8]:
# We drop the first column "Unnamed: 0" from the raw_data df:
raw_data.drop(columns = "Unnamed: 0",inplace = True)
raw_data['rating_sentiment'] = raw_data['review_rating'].apply(lambda x: 'negative' if x in ['1', '2'] else ('neutral' if x == '3' else 'positive'))

In [9]:
display(raw_data.dtypes)

review_title        object
review_rating       object
review_content      object
source              object
review_id           object
rating_sentiment    object
dtype: object

In [10]:
raw_data['rating_sentiment'] = raw_data['review_rating'].apply(lambda x: 'negative' if x in ['1', '2'] else ('neutral' if x == '3' else 'positive'))

In [11]:
raw_data.head()

,review_title,review_rating,review_content,source,review_id,rating_sentiment
0,"TIG (WIG) welder, model S-WIGMA 200PDC",5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",expondo,0,positive
1,Awesome experience,5,Awesome experience. We had to use their custom...,expondo,1,positive
2,The product arrived on time all the…,5,The product arrived on time all the assembly d...,expondo,2,positive
3,Good and fast delivery,5,Good and fast delivery of this amazing standin...,expondo,3,positive
4,Bad invoice cross-border purchase - no contact...,1,Bad invoice - DPH on invoice - shipping to slo...,expondo,4,negative


In [12]:
source_rating_df = raw_data.groupby(["source", "review_rating"]).size().reset_index(name='count')
source_rating_df

,source,review_rating,count
0,amazon_neutral,1,15
1,amazon_neutral,2,2
2,amazon_neutral,3,797
3,amazon_neutral,4,1
4,amazon_neutral,5,2
5,asos,1,1802
6,asos,2,121
7,asos,3,77
8,bank,1,1233
9,bank,2,211


In [13]:
raw_data = raw_data[~((raw_data["source"] == "bank") & (raw_data["review_rating"] == "5"))]

In [14]:
# We have 30000 rows, but for the sake of being able to perform the modelling (especially with the texblob, we will reduce the df to 12 K rows, 
# excluding the 5 star reviews from the bank df, which turn to be very similar and have less added value than the other sources

raw_data_sentiment = raw_data.groupby("review_rating").size().reset_index(name='count')
raw_data_sentiment

,review_rating,count
0,1,3688
1,2,412
2,3,1836
3,4,2199
4,5,2944


In [15]:
raw_data = raw_data.reset_index()

In [16]:
raw_data = raw_data.drop(columns=["index", "review_id"], axis=1)

In [17]:
raw_data["id"] = raw_data.index

In [18]:
raw_data.head()

,review_title,review_rating,review_content,source,rating_sentiment,id
0,"TIG (WIG) welder, model S-WIGMA 200PDC",5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",expondo,positive,0
1,Awesome experience,5,Awesome experience. We had to use their custom...,expondo,positive,1
2,The product arrived on time all the…,5,The product arrived on time all the assembly d...,expondo,positive,2
3,Good and fast delivery,5,Good and fast delivery of this amazing standin...,expondo,positive,3
4,Bad invoice cross-border purchase - no contact...,1,Bad invoice - DPH on invoice - shipping to slo...,expondo,negative,4


## Sentiment analysis

### Step 1. VADER Seniment Scoring

Step 1. VADER Sentiment Scoring
We will use NLTK's SentimentIntensityAnalyzer to get the neg/neu/pos scores of the text.

This uses a "bag of words" approach:
Stop words are removed
each word is scored and combined to a total score.

In [19]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [20]:
# Run the polarity score on the entire dataset
res = {}
for i, row in tqdm(raw_data.iterrows(), total=len(raw_data)):
    text = row['review_content'] if not pd.isna(row['review_content']) else ''  # Convert to string, handle NaN
    myid = row['id']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/11079 [00:00<?, ?it/s]

In [21]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'id'})
display(vaders.shape)
display(vaders.head())

(11079, 5)

,id,neg,neu,pos,compound
0,0,0.02,0.879,0.102,0.9091
1,1,0.00,0.552,0.448,0.9299
2,2,0.00,0.571,0.429,0.9147
3,3,0.00,0.586,0.414,0.8908
4,4,0.39,0.530,0.080,-0.7920


In [22]:
display(raw_data.shape)
result_df_raw = vaders.merge(raw_data, how='left',left_on="id",right_on="id")
display(result_df_raw.head())
result_df_raw.shape

(11079, 6)

,id,neg,neu,pos,compound,review_title,review_rating,review_content,source,rating_sentiment
0,0,0.02,0.879,0.102,0.9091,"TIG (WIG) welder, model S-WIGMA 200PDC",5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",expondo,positive
1,1,0.00,0.552,0.448,0.9299,Awesome experience,5,Awesome experience. We had to use their custom...,expondo,positive
2,2,0.00,0.571,0.429,0.9147,The product arrived on time all the…,5,The product arrived on time all the assembly d...,expondo,positive
3,3,0.00,0.586,0.414,0.8908,Good and fast delivery,5,Good and fast delivery of this amazing standin...,expondo,positive
4,4,0.39,0.530,0.080,-0.7920,Bad invoice cross-border purchase - no contact...,1,Bad invoice - DPH on invoice - shipping to slo...,expondo,negative


(11079, 10)

In [23]:
result_df_raw["sentiment_vader"] = result_df_raw["compound"].apply(lambda x: 'positive' if x >= 0.05 else ('negative' if x <= -0.05 else 'neutral'))

In [24]:
result_df_raw.tail()

,id,neg,neu,pos,compound,review_title,review_rating,review_content,source,rating_sentiment,sentiment_vader
11074,11074,0.034,0.884,0.082,0.4927,Delivery,3,HiI needed my mower delivered on a Wednesday a...,mowdirect_neutral,neutral,positive
11075,11075,0.094,0.842,0.064,-0.6021,Okay product but delivery not as advertised,3,Product is fine and works as intended but it a...,mowdirect_neutral,neutral,negative
11076,11076,0.097,0.858,0.045,-0.8012,Mower drekkly,3,Ordered mower on Monday and as I live in cornw...,mowdirect_neutral,neutral,negative
11077,11077,0.000,0.886,0.114,0.7799,Goods not received,3,"We ordered three items, the battery operated m...",mowdirect_neutral,neutral,positive
11078,11078,0.247,0.687,0.066,-0.8979,Overall good service,3,I would have given five stars as the product i...,mowdirect_neutral,neutral,negative


In [25]:
result_df_raw.review_rating.unique()

array(['5', '1', '4', '3', '2'], dtype=object)

In [26]:
result_df_raw["review_content"].isna().sum()

131

In [27]:
result_df_raw.dtypes

id                    int64
neg                 float64
neu                 float64
pos                 float64
compound            float64
review_title         object
review_rating        object
review_content       object
source               object
rating_sentiment     object
sentiment_vader      object
dtype: object

In [28]:
def count_whitespace_records(df):
    whitespace_counts = {}

    for column in df.columns:
        # Check for white spaces in the column (convert to string before checking)
        empty_values = df[column].astype(str).eq('') | df[column].astype(str).eq(' ') | df[column].astype(str).eq('  ') | df[column].astype(str).str.isspace()

        # Count the number of records with white spaces
        whitespace_count = empty_values.sum()

        # Add the column and count to the dictionary
        whitespace_counts[column] = whitespace_count

    return whitespace_counts

In [29]:
count_whitespace_records(result_df_raw)

{'id': 0,
 'neg': 0,
 'neu': 0,
 'pos': 0,
 'compound': 0,
 'review_title': 0,
 'review_rating': 0,
 'review_content': 0,
 'source': 0,
 'rating_sentiment': 0,
 'sentiment_vader': 0}

## Step 2. TextBlob

In [57]:
result_df_raw.head()

,id,neg,neu,pos,compound,review_title,review_rating,review_content,source,rating_sentiment,sentiment_vader,sentiment_textblob
0,0,0.02,0.879,0.102,0.9091,"TIG (WIG) welder, model S-WIGMA 200PDC",5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",expondo,positive,positive,positive
1,1,0.00,0.552,0.448,0.9299,Awesome experience,5,Awesome experience. We had to use their custom...,expondo,positive,positive,positive
2,2,0.00,0.571,0.429,0.9147,The product arrived on time all the…,5,The product arrived on time all the assembly d...,expondo,positive,positive,positive
3,3,0.00,0.586,0.414,0.8908,Good and fast delivery,5,Good and fast delivery of this amazing standin...,expondo,positive,positive,positive
4,4,0.39,0.530,0.080,-0.7920,Bad invoice cross-border purchase - no contact...,1,Bad invoice - DPH on invoice - shipping to slo...,expondo,negative,negative,negative


In [58]:
from textblob import TextBlob

In [59]:
def analyze_sentiment(text):
    analysis = TextBlob(text)
    # Classify polarity as positive, negative, or neutral
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

In [60]:
# Apply sentiment analysis to the 'Review Content' column
result_df_raw['sentiment_textblob'] = result_df_raw['review_content'].astype(str).apply(analyze_sentiment)

In [61]:
# Display the resulting DataFrame
display(result_df_raw.head())

,id,neg,neu,pos,compound,review_title,review_rating,review_content,source,rating_sentiment,sentiment_vader,sentiment_textblob
0,0,0.02,0.879,0.102,0.9091,"TIG (WIG) welder, model S-WIGMA 200PDC",5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",expondo,positive,positive,positive
1,1,0.00,0.552,0.448,0.9299,Awesome experience,5,Awesome experience. We had to use their custom...,expondo,positive,positive,positive
2,2,0.00,0.571,0.429,0.9147,The product arrived on time all the…,5,The product arrived on time all the assembly d...,expondo,positive,positive,positive
3,3,0.00,0.586,0.414,0.8908,Good and fast delivery,5,Good and fast delivery of this amazing standin...,expondo,positive,positive,positive
4,4,0.39,0.530,0.080,-0.7920,Bad invoice cross-border purchase - no contact...,1,Bad invoice - DPH on invoice - shipping to slo...,expondo,negative,negative,negative


In [62]:
result_df_raw.to_csv("result_df_raw.csv",index=False,sep=";")

In [63]:
#results_df.to_csv("results_for sentiment2.csv",index=False,sep=";")

In [64]:
from sklearn.metrics import classification_report

print("TextBlob:")
print(classification_report(result_df_raw['rating_sentiment'], result_df_raw['sentiment_textblob']))

print("\nVADER:")
print(classification_report(result_df_raw['rating_sentiment'], result_df_raw['sentiment_vader']))

TextBlob:
              precision    recall  f1-score   support

    negative       0.75      0.44      0.56      4100
     neutral       0.24      0.10      0.14      1836
    positive       0.60      0.92      0.72      5143

    accuracy                           0.61     11079
   macro avg       0.53      0.49      0.47     11079
weighted avg       0.59      0.61      0.56     11079


VADER:
              precision    recall  f1-score   support

    negative       0.73      0.53      0.61      4100
     neutral       0.28      0.10      0.15      1836
    positive       0.62      0.91      0.74      5143

    accuracy                           0.63     11079
   macro avg       0.55      0.51      0.50     11079
weighted avg       0.61      0.63      0.59     11079

